## web.py αρχείο
Εισαγωγή βιβλιοθηκών

In [60]:
import requests
import logging
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

Η δομή Trie


In [61]:
trie = None
final_tokens = None 
inverted_index = defaultdict(set)

class TrieNode:
    def __init__(self):
        self.children = {}
        self.doc_ids = set()

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word, doc_id):
        current = self.root
        for char in word:
            if char not in current.children:
                current.children[char] = TrieNode()
            current = current.children[char]
        current.doc_ids.add(doc_id)

    def search(self, word):
        current = self.root
        for char in word:
            if char not in current.children:
                return set()
            current = current.children[char]
        return current.doc_ids


Ορισμός βοηθητικών συναρτήσεων


In [62]:
ps = PorterStemmer()
def default_run():
    try:
        nltk.download('stopwords')
        nltk.download('punkt_tab')
        stop_words = set(stopwords.words('english'))
        print("Setup complete")
    except Exception as e:
        print("Error setting up : {e}")

# Βήμα 1 - Web Scrapping
Η ρουτίνα scrape_wikipedia(URL) έχει ως όρισμα μια ιστοσελίδα από την wikipedia και τη κάνει parse. Έπειτα, καλεί την scrape_pargraphs().

In [63]:
def scrape_wikipedia(URL) :
    r = requests.get(URL)
    page = BeautifulSoup(r.text ,"html.parser")
    scrape_paragraphs(page)

Η ρουτίνα scrape_paragraphs(page) έχει ως όρισμα το page που παράγει η scrape_wikipedia και για κάθε παράγραφο του page, κρατάει μόνο το κείμενο που θα είναι χρήσιμο για μετέπειτα ανάλυση.

In [64]:
def scrape_paragraphs(page):
    try:
        data = []
        paragraph_id = 0  # Το ID για κάθε παράγραφο
        for paragraph in page.select('p'):
            text = paragraph.getText().strip()  
            if text: 
                data.append(text)
                tokens = text.lower().split()  # Tokenize
                for token in tokens:
                    inverted_index[token].add(paragraph_id)  # Ενημερώνουμε το ευρετήριο
                paragraph_id += 1
        save_to_csv(data)
    except Exception as e:
        print(f"Error while processing paragraphs: {e}")

Η ρουτίνα save_to_csv(data) αποθηκεύει κάθε scraped παραγράφο στο αρχείο "wikipedia_data.csv"

In [65]:
def save_to_csv(data):
    try:
        df = pd.DataFrame(data,columns=["Paragraph Text"])
        df.to_csv('wikipedia_data.csv', index= False , encoding='utf-8')
        print("Data saved Successfuly!")
    except Exception as e:
        print(f"Error Saving in csv File")

# Βήμα 2 - Text Processing
Για κάθε παράγραφο που υπάρχει στο wikipedia_data.csv, γίνεται επεξεργασία με μεθόδους όπως stop-word removal, tokenize, stemming.

Αφενός, η ρουτίνα text_processing(CSVFile) ανακτά τις παραγράφους που είναι αποθηκευμένες στο csvFile και τις αποθηκεύει στην cleaned_data = [].
Αφετέρου, οι πάραγραφοι καθαρίζονται όταν καλείται η cleaning_text_and_save(text).

Η ρουτίνα cleaning_text_and_save περιέχει όλες τις εργασίες που πρέπει να γίνουν στο κείμενο.
Πρώτα, αφαιρεί όλα τα σημεία στίξης, έπειτα τις κάνει tokenize και τέλος stem.
Έπειτά, καλείται η create_inverted_index που αναλυέται στο βήμα 3.

In [66]:
def text_processing(csvFile):
    
    try:
        df = pd.read_csv(csvFile, header=None)
        cleaned_data = []

        for text in df[0]:
            if text:
                cleaned_text = cleaning_text_and_save(text)
                cleaned_data.append({cleaned_text})

        cleaned_df = pd.DataFrame(cleaned_data)
        save_new_text(cleaned_data)
    except BaseException:
        logging.exception("An exception was thrown")


def cleaning_text_and_save(text):
    
        #Stop-word removal
        stop_words = set(stopwords.words('english'))
        #Tokenize
        tokens = word_tokenize(text)
        filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
        stemmed_tokens = [ps.stem(word) for word in filtered_tokens]
        cleaned_tokens = [re.sub(r'[^a-zA-Z0-9]', '', token) for token in stemmed_tokens]
        #Clearing Text
        cleaned_text = ''.join(stemmed_tokens)
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_text)
        #Final Tokens
        global final_tokens
        final_tokens = [token for token in cleaned_tokens if token]
        create_inverted_index(final_tokens)
        

        return cleaned_text



def save_new_text(cleaned_data):
    try:
        df = pd.DataFrame(cleaned_data)
        df.to_csv('cleaned_text.csv', index= False , encoding='utf-8')
        print("Cleaned data saved Successfuly!")
    except Exception as e:
        print(f"Error Saving in csv File")

# Βήμα 3 - Inverted Index και Υλοποιήση δομής δεδομένων
Η create_inverted_index(final_tokens), παίρνει τα καθαρισμένα πλέον tokens που έβγαλε η cleaning_text_and_save και τα προσθέτει ένα ένα στο inverted index.

Η inverted index είναι τύπου defaultdict αλλά χρησιμοποιείται σαν την dict. Μόνο που η defaultdict επιστρέφει set() (κενό σύνολο) σε περιπτώση που δε βρεθεί κάποιο κλειδί στους όρους της.

Κάθε token προστίθεται στο inverted index.

Και τέλος το inverted_index εξάγεται σαν txt file για την ευκολότερη επεξεργασία του αργότερα.

In [67]:
def create_inverted_index(final_tokens):
    global inverted_index 
    inverted_index = defaultdict(set) 
    
    for idx, token in enumerate(final_tokens):
        token = token.lower()  
        if token: 
            inverted_index[token].add(idx)  
    
    with open("inverted_index.txt", 'a', encoding='utf-8') as file:
        for token, doc_ids in inverted_index.items():
            doc_ids_str = ', '.join(map(str, doc_ids))  
            file.write(f"Token: {token} | Document IDs: {doc_ids_str}\n")

Η ρουτίνα insert_inverted_index_to_trie() μετρατρέπει το inverted_index σε trie. Για κάθε όρο, γίνεται insert το id του και ο ίδιος ο όρος.

In [68]:
def insert_inverted_index_to_trie(inverted_index, trie):
    for term, doc_ids in inverted_index.items():
        for doc_id in doc_ids:
            trie.insert(term, doc_id)

Η συνάρτηση load_inverted_index_with_trie φορτώνει ένα αντεστραμμένο ευρετήριο (inverted index) από ένα αρχείο και το αποθηκεύει σε μια δομή Trie, για γρήγορη αναζήτηση λέξεων και των παραγράφων που εμφανίζονται.
Επιστρέφει ένα αντικείμενο trie, διaφορετικά None σε περίπτωση λάθους.

In [69]:
def load_inverted_index_with_trie(filename, num_paragraphs):
    trie = Trie()
    try:
        with open(filename, 'r') as file:
            for line in file:
                parts = line.strip().split(' | ')  
                token = parts[0].split(': ')[1]  
                doc_ids_str = parts[1].split(': ')[1]  
                
                doc_ids = map(int, doc_ids_str.split(', '))
                valid_doc_ids = [doc_id for doc_id in doc_ids if 0 <= doc_id < num_paragraphs]  
                for doc_id in valid_doc_ids:
                    trie.insert(token, doc_id)  
    except Exception as e:
        print(f"Error loading the inverted index: {e}")
        return None
    
    return trie

# Bήμα 4 : Μηχανή αναζήτησης 
Σε αυτό το βήμα, έχουν υλοποιηθεί συναρτήσεις που είναι αναγκαιές για την web διεπάφη.
Το αρχείο της web διεπαφής θα αναλυθεί πιο κάτω.

Η συνάρτηση search_in_inverted_index(token, inverted_index) επιστρέφει τα ID στα οποία υπάρχουν το token. Διαφορετικά γυρνάει κενό set().

In [70]:
def search_in_inverted_index(token, inverted_index):
    return set(inverted_index.get(token, []))

Η συνάρτηση query_processing(query,trie,debug=False) είναι μια ρουτίνα η οποία επεξεργάζεται το query που δίνει ο user.

Αφενός το query γίνεται tokenized.

To query μπορεί να έχει τη μορφή :

word1

word1 AND word2

word1 OR word2

NOT word1

word1 OR word2 AND word3 NOT word4

  for token in tokens:
        if debug:
            print(f"Processing token: {token}")
        
        if token == "and":
            current_operator = "AND"
            continue
        elif token == "or":
            current_operator = "OR"
            continue
        elif token == "not":
            current_operator = "NOT"
            continue
Σε αυτό το κομμάτι διαχωρίζεται το token από τις λογικές λέξεις. 

Έπειτα γίνεται μια αναζήτηση στο trie για κάθε token και αποθηκεύεται στη μεταβλητή doc_ids τα σύνολα όπου υπάρχουν τα tokens.

Για κάθε token στο σύνολο doc_ids,  
 -> Aν ο τελεστής είναι "AND", το result_set επικαλύπτεται με τα κοινά doc_ids των δυο συνόλων (παλιό result_set και doc_ids)

 -> Aν ο τελεστής είναι "OR", το result_set επεκτείνεται για να περιλάβει όλα τα doc_ids που εμφανίζονται είτε στο παλιό σύνολο είτε στο νέο doc_ids.

 -> Αν ο τελεστής είναι "NOT", το result_set αφαιρεί τα doc_ids που εμφανίζονται στο doc_ids από το υπάρχον σύνολο.

 Επιστρέφεται το result_set έπειτα από την επεξεργασία.

In [71]:
def query_processing(query, trie, debug=False):
    query = query.strip()
    tokens = query.lower().split()
    
    if debug:
        print("Tokens from query:", tokens)
    
    tokens_set = set(tokens)
    
    if debug:
        print("Unique tokens set:", tokens_set)
    
    result_set = None
    current_operator = "AND"
    
    for token in tokens:
        if debug:
            print(f"Processing token: {token}")
        
        if token == "and":
            current_operator = "AND"
            continue
        elif token == "or":
            current_operator = "OR"
            continue
        elif token == "not":
            current_operator = "NOT"
            continue
        
        doc_ids = trie.search(token)  # Αναζητούμε το token στο trie
        
        if debug:
            print(f"Found doc_ids for '{token}': {doc_ids}")
        
        if result_set is None:
            result_set = doc_ids
        elif current_operator == "AND":
            result_set &= doc_ids
        elif current_operator == "OR":
            result_set |= doc_ids
        elif current_operator == "NOT":
            result_set -= doc_ids
    return result_set

H συνάρτηση search_in_invered_index αναζητά ένα token στο inverted_index.
Αν το βρεί, γυρνάει τη θέση του.
Αν όχι, γυρνάει κενό set().

In [72]:
def search_in_inverted_index(token,inverted_index):
    token = token.lower()
    print("Token in function :", token)
    if token in inverted_index:
        return inverted_index[token]
    else:
        return set()

Εδώ υλοποιούνται  οι αλγόριθμοι TF-IDF και BM25. 

Η συνάρτηση search_tfidf χρησιμοποιεί την τεχνική TF-IDF (Term Frequency-Inverse Document Frequency) για να αναζητήσει τις παραγράφους που είναι πιο συναφείς με ένα δοσμένο ερώτημα (query). Αρχικά, μετατρέπει όλες τις παραγράφους και το query σε χρησιμοποιώντας τον TfidfVectorizer, δημιουργώντας έναν πίνακα χαρακτηριστικών για κάθε παράγραφο και ένα vector για το query. Στη συνέχεια, υπολογίζει την ομοιότητα συνημιτόνου (cosine similarity) μεταξύ του query και κάθε παραγράφου για να μετρήσει πόσο κοντά είναι το query με κάθε παράγραφο. Οι παράγραφοι ταξινομούνται με βάση την ομοιότητά τους με το query, επιστρέφοντας τις πιο σχετικές πρώτες. Επιστρέφει μια λίστα με παραγράφους που έχουν θετική ομοιότητα με το query, ταξινομημένες από τις πιο σχετικές στις λιγότερο σχετικές.

In [73]:
def search_tfidf(query,paragraphs,trie):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(paragraphs)
    query_vector = vectorizer.transform([query])
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix).flatten()
    ranked_indices = similarity_scores.argsort()[::-1]
    ranked_paragraphs = [paragraphs[i] for i in ranked_indices if similarity_scores[i] > 0]

    return ranked_paragraphs

Η συνάρτηση search_bm25 χρησιμοποιεί τον αλγόριθμο BM25 (Best Matching 25) για να αναζητήσει τις παραγράφους που είναι πιο συναφείς με το δοσμένο query. Αρχικά, η συνάρτηση μετατρέπει κάθε παράγραφο και το query σε λίστες από λέξεις (tokens), χρησιμοποιώντας την συνάρτηση word_tokenize από την βιβλιοθήκη nltk, και μετατρέπει τα κείμενα σε μικρούς χαρακτήρες (lowercase) για να εξασφαλίσει ότι η αναζήτηση είναι case-insensitive. Στη συνέχεια, δημιουργεί ένα αντικείμενο του τύπου BM25Okapi που υπολογίζει τα σκορ BM25 για κάθε παράγραφο, με βάση τη συχνότητα εμφάνισης των λέξεων του query σε κάθε παράγραφο και τη συχνότητα εμφάνισης των λέξεων στο σύνολο των παραγράφων. Τα σκορ αυτά δείχνουν πόσο συναφής είναι κάθε παράγραφος με το query. Οι παράγραφοι ταξινομούνται κατά φθίνουσα σειρά ομοιότητας (βάσει των σκορ BM25), και επιστρέφονται οι παράγραφοι με τα υψηλότερα σκορ, που υποδηλώνουν τη μεγαλύτερη συνάφεια με το query.

In [74]:
def search_bm25(query, paragraphs, trie): 
    tokenized_paragraphs = [word_tokenize(paragraph.lower()) for paragraph in paragraphs]
    tokenized_query = word_tokenize(query.lower())
    bm25 = BM25Okapi(tokenized_paragraphs)
    scores = bm25.get_scores(tokenized_query)
    ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    ranked_paragraphs = [paragraphs[i] for i in ranked_indices if scores[i] > 0]
    
    return ranked_paragraphs

# Βήμα 5 - Αξιολόγηση συστήματος
Η συνάρτηση calculate_metrics υπολογίζει τρεις βασικούς δείκτες απόδοσης για την αναζήτηση:

Precision :Το ποσοστό των ανακτηθέντων εγγράφων που είναι πραγματικά σχετικά. Υπολογίζεται ως το πηλίκο του αριθμού των σχετικών εγγράφων που ανακτήθηκαν προς τον αριθμό των εγγράφων που ανακτήθηκαν.
Recall : Το ποσοστό των σχετικών εγγράφων που ανακτήθηκαν σωστά. Υπολογίζεται ως το πηλίκο του αριθμού των σχετικών εγγράφων που ανακτήθηκαν προς τον αριθμό των συνολικών σχετικών εγγράφων.
F1-Score: Η αρμονική μέση της ακρίβειας και της ανάκλησης. Είναι χρήσιμος όταν υπάρχει μια αδικία μεταξύ της ακρίβειας και της ανάκλησης, και προσφέρει μια συνολική μέτρηση της απόδοσης του συστήματος.
Η συνάρτηση χρησιμοποιεί τις εξής διαδικασίες:

Δημιουργεί δυαδικούς πίνακες (arrays) για την ακρίβεια (retrieved) και τη σχετικότητα (relevant) των εγγράφων, που περιέχουν 1 για κάθε σχετικό έγγραφο και 0 για τα υπόλοιπα.
Υπολογίζει τα σκορ χρησιμοποιώντας τις βασικές μαθηματικές σχέσεις μεταξύ των ακρίβειας, ανάκλησης και F1-Score.

In [75]:
def calculate_metrics(retrieved_docs, relevant_docs):
    if not retrieved_docs and not relevant_docs:
        return 0.0, 0.0, 0.0

    all_docs = retrieved_docs.union(relevant_docs)
    if not all_docs:
        return 0.0, 0.0, 0.0
    
    
    retrieved = np.array([1 if doc in retrieved_docs else 0 for doc in range(max(retrieved_docs.union(relevant_docs)) + 1)])
    relevant = np.array([1 if doc in relevant_docs else 0 for doc in range(max(retrieved_docs.union(relevant_docs)) + 1)])
    
    precision = np.sum(retrieved & relevant) / np.sum(retrieved) if np.sum(retrieved) > 0 else 0
    recall = np.sum(retrieved & relevant) / np.sum(relevant) if np.sum(relevant) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

H συνάρτηση  calculate_map υπολογίζει την Μέση Ακρίβεια (MAP) για την αναζήτηση. Η MAP υπολογίζει την ακρίβεια σε κάθε θέση της λίστας αποτελεσμάτων, αλλά μόνο για τα σχετικά έγγραφα, και στη συνέχεια υπολογίζει τη μέση ακρίβεια αυτών των αποτελεσμάτων για όλα τα ερωτήματα:

Για κάθε έγγραφο στη λίστα των ανακτηθέντων εγγράφων, αν το έγγραφο είναι σχετικό, υπολογίζεται η ακρίβεια στο σημείο (δηλαδή ο αριθμός των σχετικών εγγράφων που έχουν βρεθεί μέχρι αυτήν την θέση, δια του συνολικού αριθμού εγγράφων που έχουν ανακτηθεί μέχρι εκεί).
Η MAP είναι η μέση ακρίβεια για όλα τα ερωτήματα.

In [76]:
def calculate_map(retrieved_docs, relevant_docs):
    if not relevant_docs:
        return 0.0
    avg_precision = 0
    hits = 0
    for i, doc in enumerate(retrieved_docs, 1):
        if doc in relevant_docs:
            hits += 1
            avg_precision += hits / i
    return avg_precision / len(relevant_docs)

Η συνάρτηση evaluate_systems αξιολογεί το σύστημα αναζήτησης για ένα σύνολο από ερωτήματα:

queries: Τα ερωτήματα που θέλουμε να αξιολογήσουμε.

relevant_docs_set: Ένα σύνολο με τα σχετικά έγγραφα για κάθε ερώτημα.

trie: Η δομή δεδομένων trie που χρησιμοποιείται για αναζήτηση.

paragraphs: Οι παράγραφοι που περιέχουν τα έγγραφα προς αναζήτηση.

In [77]:
def evaluate_system(queries, relevant_docs_set, trie, paragraphs):
    results = []
    for query in queries:
        print(f"Evaluating query: {query}")
        
        retrieved_docs = query_processing(query, trie) or set()
        
        relevant_docs = relevant_docs_set.get(query, set())
        
      
        precision, recall, f1 = calculate_metrics(retrieved_docs, relevant_docs)
        map_score = calculate_map(list(retrieved_docs), relevant_docs)
        
        results.append({
            "query": query,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "map": map_score
        })
        print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}, MAP: {map_score:.2f}")
    return results

Οι getters :

In [78]:
#getters    
def get_tokens():
    global final_tokens
    return final_tokens


def get_inverted_index():
    global inverted_index
    return inverted_index

Η main και οι λειτουργίες της :

Η default_run κατεβάζει τα πακέτα, καθορίζει το setup του stemmer κτλ.

H scrape_wikipedia ξεκινάει όλη την διαδικασία της ανάκτησης πληροφορίας για το δωσμένο link.

Η text_processing ξεκινάει την επεξεργασία κειμένου για το file "wikipedia_data.csv".


In [79]:
def main():
    default_run() #Περιλαμβάνει το κατέβασμα των πακέτων, το setup του stemmer κτλ
    scrape_wikipedia("https://en.wikipedia.org/wiki/cristiano_ronaldo")  #Bήμα 1
    text_processing("wikipedia_data.csv") #Βήμα 2
    global inverted_index
    inverted_index = get_inverted_index()
    
    

if __name__ == "__main__":
    main()
    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lyprandos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Lyprandos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Setup complete
Data saved Successfuly!
Cleaned data saved Successfuly!


## app.py - Η web διεπαφή.

Ο παρακάτω κώδικας υλοποιεί μια εφαρμογή αναζήτησης που εκτελείται μέσω του web framework Flask. Η εφαρμογή επιτρέπει στους χρήστες να εισάγουν ερωτήματα αναζήτησης (queries) και να επιλέξουν διάφορους αλγόριθμους αναζήτησης (Boolean Retrieval, TF-IDF, και Okapi BM25) για να ανακτήσουν σχετικά παραγράμματα από μια βάση δεδομένων (ένα CSV αρχείο με παραγράφους από το Wikipedia). Ας το δούμε βήμα-βήμα:

Αρχικοποίηση Flask και φόρτωση δεδομένων
Flask app: Ο κώδικας δημιουργεί μια εφαρμογή Flask με την εντολή app = Flask(__name__), η οποία παρέχει τη δυνατότητα για web περιβάλλον και εξυπηρετεί αιτήματα HTTP.
load_paragraphs(): Αυτή η συνάρτηση διαβάζει τις παραγράφους από το αρχείο wikipedia_data.csv και τις αποθηκεύει σε μια λίστα paragraphs.
load_inverted_index(): Η συνάρτηση διαβάζει το αρχείο inverted_index.txt για να φορτώσει το αναστραμμένο ευρετήριο (inverted index), το οποίο είναι μια δομή δεδομένων που συνδέει λέξεις (tokens) με τα έγγραφα (παράγραφοι) που τις περιέχουν.
load_inverted_index_with_trie(): Φορτώνει επίσης το αναστραμμένο ευρετήριο χρησιμοποιώντας τη δομή δεδομένων Trie, η οποία επιτρέπει γρήγορη αναζήτηση.
Λειτουργία της αναζήτησης
Η συνάρτηση search() είναι η κύρια διαδρομή (route) της εφαρμογής και εξυπηρετεί το αίτημα αναζήτησης. Υποστηρίζει τις μεθόδους GET και POST.
Όταν η σελίδα φορτώνει για πρώτη φορά (GET), φορτώνει τις παραγράφους και το αναστραμμένο ευρετήριο, και ετοιμάζει την εφαρμογή για αναζητήσεις.
Όταν ο χρήστης υποβάλει μια αναζήτηση (POST), η εφαρμογή παίρνει το query (ερώτημα) και τον αλγόριθμο που επέλεξε ο χρήστης για την αναζήτηση. Στη συνέχεια, η εφαρμογή αναζητά τα σχετικά παραγράφους χρησιμοποιώντας τον επιλεγμένο αλγόριθμο. Επιλογή αλγορίθμου και αναζήτηση
Υπάρχουν τρεις αλγόριθμοι αναζήτησης που υποστηρίζονται από την εφαρμογή:

Boolean Retrieval: Χρησιμοποιείται για την αναζήτηση με βάση λογικούς τελεστές (AND, OR, NOT). Ο χρήστης μπορεί να αναζητήσει λέξεις και να βρει τα σχετικά έγγραφα μέσω του αναστραμμένου ευρετηρίου.
TF-IDF (Term Frequency-Inverse Document Frequency): Αυτή η μέθοδος χρησιμοποιεί τις συχνότητες λέξεων σε σχέση με το πόσο κοινές είναι σε όλο το σύνολο των παραγράφων για να υπολογίσει την σχετικότητα κάθε παραγράφου για το ερώτημα.
Okapi BM25: Αυτή είναι μια στατιστική μέθοδος αναζήτησης που υπολογίζει την σχετικότητα κάθε παραγράφου με βάση την εμφάνιση των λέξεων και ένα σύνολο ρυθμιζόμενων παραμέτρων.
Αξιολόγηση Απόδοσης
Όταν ολοκληρωθεί η αναζήτηση, η εφαρμογή υπολογίζει την απόδοση του συστήματος αναζήτησης μέσω μετρικών αξιολόγησης όπως:
Precision: Το ποσοστό των ανακτηθέντων εγγράφων που είναι πραγματικά σχετικά.
Recall: Το ποσοστό των σχετικών εγγράφων που ανακτήθηκαν.
F1-Score: Ο αρμονικός μέσος των precision και recall.
MAP (Mean Average Precision): Ο μέσος όρος της ακρίβειας σε κάθε σημείο της λίστας αποτελεσμάτων.
Αυτές οι μετρήσεις υπολογίζονται χρησιμοποιώντας τις συναρτήσεις calculate_metrics και calculate_map από το module web.
Αποστολή Αποτελεσμάτων στην Σελίδα
Μετά την επεξεργασία της αναζήτησης και την αξιολόγηση της απόδοσης, τα αποτελέσματα της αναζήτησης (τα σχετικά παραγράμματα) και οι μετρικές απόδοσης (precision, recall, F1, MAP) επιστρέφονται στον χρήστη μέσω της λειτουργίας render_template_string.
Το αποτέλεσμα εμφανίζεται στην ιστοσελίδα και περιλαμβάνει μια λίστα με τις παραγράφους που βρέθηκαν και τα αντίστοιχα αποτελέσματα των μετρικών.
Εκκίνηση του Web Server
Στο τέλος, η εφαρμογή εκκινεί τον web server με την εντολή app.run(debug=True), επιτρέποντας στην εφαρμογή να λειτουργεί και να ανταποκρίνεται στα αιτήματα αναζήτησης από το χρήστη.

In [80]:
from flask import Flask, request, render_template_string
import web
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

app = Flask(__name__)
final_tokens = web.get_tokens() 

def load_paragraphs():
    paragraphs = []
    with open('wikipedia_data.csv', 'r', encoding='utf-8') as file:
        for line in file:
            paragraphs.append(line.strip())
    return paragraphs

#def search_paragraphs(result_set, inverted_index, paragraphs):
    #result_paragraphs = []

    for doc_id in result_set:
        if 0 <= doc_id < len(paragraphs):
            result_paragraphs.append(paragraphs[doc_id])
        else:
            print(f"Invalid doc_id: {doc_id} is out of range.")
    
    if not result_paragraphs:
        return ["No valid paragraphs found."]
    return result_paragraphs



def load_inverted_index(filename, num_paragraphs):
    inverted_index = defaultdict(set)
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split(' | ')  
            token = parts[0].split(': ')[1] 
            doc_ids_str = parts[1].split(': ')[1]  
            
            doc_ids = map(int, doc_ids_str.split(', '))
            
            for doc_id in doc_ids:
                if 0 <= doc_id < num_paragraphs:  
                    inverted_index[token].add(doc_id)
    return inverted_index




@app.route("/", methods=["GET", "POST"])
def search():
    paragraphs = load_paragraphs()
    num_paragraphs = len(paragraphs)
    inverted_index = load_inverted_index('inverted_index.txt',num_paragraphs)  
    trie = web.load_inverted_index_with_trie('inverted_index.txt', num_paragraphs)
    result = None
    query = ""
    algorithm = "boolean" #default algorithm 
    queries = {"ronaldo","cristiano","ball","instagram","cr7","goal","scorer","foot","messi","cup","champion","legend"}
    retrieved_docs = set()
    relevant_docs_set = {}
    for query in queries:
        if query in inverted_index:
            relevant_docs_set[query] = inverted_index[query]
        else:
            relevant_docs_set[query] = set()  
    print(relevant_docs_set)

    if request.method == "POST":
        
        query = request.form.get("query", "").strip()
        algorithm = request.form.get("algorithm","boolean")
        
        if algorithm == "boolean":
            print(algorithm)
            retrieved_docs = web.query_processing(query, trie, debug=False)
            result_set = web.query_processing(query, trie, debug=False)
            result = [paragraphs[doc_id] for doc_id in result_set if 0 <= doc_id < len(paragraphs)]
        elif algorithm == "tfidf":
            print(algorithm)
            retrieved_docs = set(range(len(paragraphs)))
            result = web.search_tfidf(query,paragraphs,trie)
        elif algorithm == "bm25":
            print(algorithm)
            retrieved_docs = set(range(len(paragraphs)))
            result = web.search_bm25(query,paragraphs,trie) 

        relevant_docs = relevant_docs_set.get(query, set())
        precision, recall, f1 = web.calculate_metrics(retrieved_docs, relevant_docs)
        map_score = web.calculate_map(list(retrieved_docs), relevant_docs)

        print(f"Query: {query}")
        print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}, MAP: {map_score:.2f}")

    return render_template_string("""
    <h1>Search Paragraphs</h1>
    <form method="POST">
    <input type="text" name="query" placeholder="Enter a word to search" value="{{ query }}">
    <label for="algorithm">Choose an algorithm:</label>
    <select name="algorithm">
        <option value="boolean">Boolean Retrieval</option>
        <option value="tfidf">TF-IDF (VSM)</option>
        <option value="bm25">Okapi BM25</option>
    </select>
    <button type="submit">Search</button>
</form>
    
    {% if result %}
        <h2>Search Results for "{{ query }}":</h2>
        <ul>
            {% if result == "No words found." %}
                <li>No paragraphs found for the given query.</li>
            {% else %}
                {% for paragraph in result %}
                    <li>{{ paragraph }}</li>
                {% endfor %}
            {% endif %}
        </ul>
    {% endif %}
    """, query=query, result=result)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\Lyprandos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Ενδεικτικά τρεξίματα :
Τα παραδείγματα θα είναι σε μορφή text διότι ανακτούνται από το web.


1.  user's query : ronaldo

terminal output : 

boleean

result set {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 
38, 39, 40, 41, 42, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 63, 64, 66, 68, 69, 71, 74, 76, 77, 78, 79, 81, 82, 84, 87, 88, 89, 91, 92, 93, 94, 97, 103}

Query: ronaldo

Precision: 1.00, Recall: 1.00, F1-Score: 1.00, MAP: 1.00

2. user's query : ronaldo and cr7

terminal output :

boolean

result set {40, 5, 59, 21}

Query: ronaldo and  cr7

Precision: 0.00, Recall: 0.00, F1-Score: 0.00, MAP: 0.00

3. user's query : cup

terminal output :

tfidf

Query: cup

Precision: 0.20, Recall: 1.00, F1-Score: 0.34, MAP: 0.42

4. user's query : instagram

terminal output : 

bm25

Query: instagram

Precision: 0.03, Recall: 1.00, F1-Score: 0.06, MAP: 0.06
